# 목적

> 특정 지점에서 각 상품의 판매액을 예측하는 모델을 만들어서, 판매액 증진에 핵심적인 역할을 하는 속성들을 파악하여 영업팀에게 액션플랜 제안하고자 함

  * Item_Idenfitifer : 상품 ID 식별자 (Unique)
  * Item_weight : 상품 무게
  * Item_Fat_Content : 상품이 저지방인지 아닌지 여부
  * Item_Visibility : 특정 상품에게 주어진 상품이 지점에 진열된 면적의 비율
  * Item_Type : 상품이 속해있는 범주
  * Item_MRP : 상품의 최대 소매 가격 (Maximum Retail Price)
  * Outlet_Identifier : 지점 ID 식별자 (Unique)
  * Outlet_Establishment_Year : 지점이 설립된 년도
  * Outlet_Size : 차지하고있는 토지 면적 관점에서 지점의 크기
  * Outlet_Location_Type : 지점이 위치한 도시의 유형
  * Outlet_Type : 지점이 grocery store인지 supermarket의 한 유형인지 여부
  * Item_Outlet_Sales : 특정 지점의 상품 판매액. **예측하고자 하는 결과 변수**

In [ ]:
import numpy as np
import pandas as pd

# 시각화를 위한 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt

# Linear Regression model을 만들기 위한 라이브러리
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 데이터 스케일링을 위한 라이브러리
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

# 데이터 로드

In [ ]:
train_df = pd.read_csv('./data-sales_prediction/Train.csv')
test_df = pd.read_csv('./data-sales_prediction/Test.csv')

In [ ]:
train_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


Item_Identifier 컬럼과 Outlet_Identifier 컬럼만 ID 변수라서, 종속변수인 "Item Outlet Sales"를 예측하는데 어떤 정보를 주지 않아서 데이터셋으로 부터 제거함

In [ ]:
train_df = train_df.drop(["Item_Identifier", "Outlet_Identifier"], axis = 1)
test_df = test_df.drop(["Item_Identifier", "Outlet_Identifier"], axis = 1)

In [ ]:
train_df.info()

  * 학습데이터는 8523개 관측치와 10개의 컬럼을 가지고 있음
  * Item_Weight와 Outlet_Size는 결측치를 가지고 있음
  * Item_Fat_Content와 Item_Type, Outlet_Size, Outlet_Location_Type, Outlet_Type은 명목형 변수임.
  * Item_Weight와 Item Visibility와 Item MRP 와 Outlet_Establish_Year는 숫자형 변수임

결측치 비율

In [ ]:
(train_df.isnull().sum()/ train_df.shape[0]) * 100

In [ ]:
def split_column_by_type(dataframe: pd.DataFrame):
    cate_list, num_list = [], []
    for col in dataframe.columns:
        if (dataframe[col].dtypes == float) :
            num_list.append(col)
        else:
            if dataframe[col].dtype.char in np.typecodes['AllInteger']:
                num_list.append(col)
            else:
                cate_list.append(col)
    print(f'categorical columns : {cate_list} \nnumerical columns : {num_list}')
    return cate_list, num_list

categorical_column_list, numerical_column_list = split_column_by_type(train_df)

# 단변량 분석 (Univariate Analysis)

In [ ]:
for col in train_df[categorical_column_list].columns:
    print(col, train_df[col].nunique())

In [ ]:
fig, axes = plt.subplots(3,2, figsize = (18, 25))
fig.suptitle("Bar plot for all categorical variables in the dataset")

sns.countplot(ax = axes[0,0], x = 'Item_Fat_Content', data = train_df, color = 'lightblue'
              , order = train_df['Item_Fat_Content'].value_counts().index)

sns.countplot(ax = axes[0,1], x = 'Item_Type', data = train_df, color = 'lightblue'
              , order = train_df['Item_Type'].value_counts().index)

axes[0,1].tick_params(axis='x', rotation=30, labelsize = 8)

sns.countplot(ax = axes[1,0], x = 'Outlet_Establishment_Year', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Establishment_Year'].value_counts().index)

sns.countplot(ax = axes[1,1], x = 'Outlet_Size', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Size'].value_counts().index)

sns.countplot(ax = axes[2,0], x = 'Outlet_Location_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Location_Type'].value_counts().index)

sns.countplot(ax = axes[2,1], x = 'Outlet_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Type'].value_counts().index)

  * Low Fat를 LF나, low fat로 Regular를 reg로 표기하는 데이터 오류가 있어 보임 -> 핸들링 필요
  * Outlet Size는 Medium이 가장 많고, Small, High 순
  * Outlet Location Type에서는 Tier 3 값이 가장 많고, Tier 2, Tier 1 순
  * Outlet Size와 Outlet Location 두 컬럼을 결합하여 Tier 1 도시 유형은 갯수가 작으나 지점 크기가 클지 예상해볼 수 있고, Tier 3 과 Tier 2 에 많은 양의 아웃렛이 있기 때문에 Outlet 크기는 Medium아니 Small일 수 있을것 같다고 예상해볼 수 있을것 같음
  * Outlet Type에서는 다수가 Supermarket Type 1 유형이고, 나머지 Grocery Store과 Supermarket Type3 와 Supermarket Type2 는 비슷함

In [ ]:
fig= plt.figure(figsize = (18, 6))

sns.countplot(x = 'Item_Type', data = train_df, color = 'lightblue', order = train_df['Item_Type'].value_counts().index);

plt.xticks(rotation = 45);

  * 아이템 유형은 과일과 야채, 스낵류 가 가장 많고, 가정용품이 뒤따른다.

In [ ]:
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].map(lambda x: 'Low Fat' if x == 'low fat' or x == 'LF' else x)

train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].map(lambda x: 'Regular' if x == 'reg' else x)

In [ ]:
test_df['Item_Fat_Content'] = test_df['Item_Fat_Content'].map(lambda x: 'Low Fat' if x == 'low fat' or x == 'LF' else x)

test_df['Item_Fat_Content'] = test_df['Item_Fat_Content'].map(lambda x: 'Regular' if x == 'reg' else x)

numerical data 분석은 histogram으로 분석

numerical columns : ['Item_Weight', 'Item_Visibility', 'Item_MRP', ~~'Item_Outlet_Sales'~~]

In [ ]:
fig, axes = plt.subplots(3,2, figsize = (18, 25))
fig.suptitle("Bar plot for all categorical variables in the dataset")

sns.countplot(ax = axes[0,0], x = 'Item_Fat_Content', data = train_df, color = 'lightblue'
              , order = train_df['Item_Fat_Content'].value_counts().index)

sns.countplot(ax = axes[0,1], x = 'Item_Type', data = train_df, color = 'lightblue'
              , order = train_df['Item_Type'].value_counts().index)

axes[0,1].tick_params(axis='x', rotation=30, labelsize = 8)

sns.countplot(ax = axes[1,0], x = 'Outlet_Establishment_Year', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Establishment_Year'].value_counts().index)

sns.countplot(ax = axes[1,1], x = 'Outlet_Size', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Size'].value_counts().index)

sns.countplot(ax = axes[2,0], x = 'Outlet_Location_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Location_Type'].value_counts().index)

sns.countplot(ax = axes[2,1], x = 'Outlet_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Type'].value_counts().index)

  * Item Weight 변수는 Uniform 분포에 가까워 보인다. 결측치를 보정할 때, 분포가 변화하면 안된다는 점을 기억해두자.
  * Item Visibility 변수는 오른쪽으로 치우친 분포다. 이것은 특정 상품의 디스플레이된 면적의 퍼센테이지가 다른 아이템들보다 높다는 것을 의미한다.
  * Item MRP 변수는 다봉 정규분포 에 근사하는 것으로 보인다.

# 이변량 분석 (Bivariate Analysis)

이제 학습 데이터 셋에서 독립변수와 종속변수간의 강한 관계가 있는지, 서로서로는 연관되어 있는지 이해할 수 있는 이변량 분석을 수행해보자.

Outlet Establishment Year는 시간과 관련된 요소이기 때문에, line plot으로 지점 설립 년도와 아이탬 판매액 관계를 분석해볼 것임.

In [ ]:
fig = plt.figure(figsize = (18,6))
sns.lineplot(x = 'Outlet_Establishment_Year', y = 'Item_Outlet_Sales', data = train_df, errorbar = ("se", 3), estimator = 'mean')
# estimator : 같은 x축 수준에서 y 변수의 집계 방법, None 이면 모든 관측치를 가지고 옴

  * 평균 판매액은 거의 매년 일정한 수준이고, 시간이 지남에 따라 증감 추이가 보이지 않음. 그래서 모델링 관점에서 판매액 예측에 좋은 예측치가 되지 않을 수 있음
  * 1998년에는 평균 판매액이 확 감소하였는데, 데이터에 포함되지 않는 외부요인이 때문일 수도 있음

estimator = 'mean'은 아래의 값을 라인플랏으로 그린 것

In [ ]:
train_df.groupby('Outlet_Establishment_Year')['Item_Outlet_Sales'].mean()

estimator = None 은 max 값을 라인 플랏으로 이은 것

In [ ]:
# sns.lineplot(x = 'Outlet_Establishment_Year', y = 'Item_Outlet_Sales', data = train_df, errorbar = ("se", 3), estimator = None)

In [ ]:
# pd.set_option('display.float_format', '{:.2f}'.format)
# train_df.groupby('Outlet_Establishment_Year')['Item_Outlet_Sales'].max()

다음은 변수간 상관관계를 확인해볼 것임. 어떤 numerical 변수가 target 변수와 관련이 있는지, 다중공선성이 있는지 어떤 독립변수끼리 서로 상관관계가 있는지 확인해 볼 수 있음

In [ ]:
fig = plt.figure(figsize = (18,6))
sns.lineplot(x = 'Outlet_Establishment_Year', y = 'Item_Outlet_Sales', data = train_df, errorbar = ("se", 3), estimator = 'mean')
# estimator : 같은 x축 수준에서 y 변수의 집계 방법, None 이면 모든 관측치를 가지고 옴

  * 오직 상품 최대판매가격만 종속변수인 판매액에 선형관계가 있음을 확인할 수 있음.
  * 나머지는 변수들간에 강한 양의 또는 음의 상관관계가 보이지 않음

이제 이변량 변수들의 scattor plot을 그려보아서, 독립변수와 종속변수 간의 관계를 확인해 볼것임

In [ ]:
fig, axes = plt.subplots(3,2, figsize = (18, 25))
fig.suptitle("Bar plot for all categorical variables in the dataset")

sns.countplot(ax = axes[0,0], x = 'Item_Fat_Content', data = train_df, color = 'lightblue'
              , order = train_df['Item_Fat_Content'].value_counts().index)

sns.countplot(ax = axes[0,1], x = 'Item_Type', data = train_df, color = 'lightblue'
              , order = train_df['Item_Type'].value_counts().index)

axes[0,1].tick_params(axis='x', rotation=30, labelsize = 8)

sns.countplot(ax = axes[1,0], x = 'Outlet_Establishment_Year', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Establishment_Year'].value_counts().index)

sns.countplot(ax = axes[1,1], x = 'Outlet_Size', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Size'].value_counts().index)

sns.countplot(ax = axes[2,0], x = 'Outlet_Location_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Location_Type'].value_counts().index)

sns.countplot(ax = axes[2,1], x = 'Outlet_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Type'].value_counts().index)

In [ ]:
fig, axes = plt.subplots(3,2, figsize = (18, 25))
fig.suptitle("Bar plot for all categorical variables in the dataset")

sns.countplot(ax = axes[0,0], x = 'Item_Fat_Content', data = train_df, color = 'lightblue'
              , order = train_df['Item_Fat_Content'].value_counts().index)

sns.countplot(ax = axes[0,1], x = 'Item_Type', data = train_df, color = 'lightblue'
              , order = train_df['Item_Type'].value_counts().index)

axes[0,1].tick_params(axis='x', rotation=30, labelsize = 8)

sns.countplot(ax = axes[1,0], x = 'Outlet_Establishment_Year', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Establishment_Year'].value_counts().index)

sns.countplot(ax = axes[1,1], x = 'Outlet_Size', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Size'].value_counts().index)

sns.countplot(ax = axes[2,0], x = 'Outlet_Location_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Location_Type'].value_counts().index)

sns.countplot(ax = axes[2,1], x = 'Outlet_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Type'].value_counts().index)

  * Item Weight와 Item Outlet Sales는 완전히 Random 관계가 있음을 보여준다. Item_Weight와 Item_outlet_Sales 간에는 어떤 관계도 없다.

  * Item_Visibility와 Item_outlet_Sales는 이들 사이에는 강한 관계가 없지만, Item_Visibility가 0.19정도 되면 판매액은 감소되는 패턴이 있는것을 보여준다.

이는 관리자가 자주 판매되지 않는 품목에 대하여 더 많이 보여지도록 위치해놓고, 가시성이 향상되면 매출이 증가할 것이라고 생각했기 때문일 수 있다.

이포인트는 high visibility 인지 low visibility인지 구분하는 범주형 변수를 만들어도 좋을것 같다.

  * Item MRP와 Item Outlet Sales간에는 명백하게 양의 상관관계가 있다. Item MRP는 판매액을 예측하는데 좋은 변수가 될 수 있을것 같다.

# 결측치 처리

Item_Weight 컬럼의 결측치를 처리할 것인데, 결측치를 평균값이나 중앙값으로 대체할 수 있고, knn과 같은 알고리즘으로 대체할 수 있을 것이다.

왜 결측치일까?

In [ ]:
def plot_categorical_column(df):
    fig, axes = plt.subplots(3,2, figsize = (18, 25))
    fig.suptitle("Bar plot for all categorical variables in the dataset")

    sns.countplot(ax = axes[0,0], x = 'Item_Fat_Content', data = df, color = 'lightblue'
                  , order = df['Item_Fat_Content'].value_counts().index)

    sns.countplot(ax = axes[0,1], x = 'Item_Type', data = df, color = 'lightblue'
                  , order = df['Item_Type'].value_counts().index)

    axes[0,1].tick_params(axis='x', rotation=30, labelsize = 8)

    sns.countplot(ax = axes[1,0], x = 'Outlet_Establishment_Year', data = df, color = 'lightblue'
                  , order = df['Outlet_Establishment_Year'].value_counts().index)

    sns.countplot(ax = axes[1,1], x = 'Outlet_Size', data = df, color = 'lightblue'
                  , order = df['Outlet_Size'].value_counts().index)

    sns.countplot(ax = axes[2,0], x = 'Outlet_Location_Type', data = df, color = 'lightblue'
                  , order = df['Outlet_Location_Type'].value_counts().index)

    sns.countplot(ax = axes[2,1], x = 'Outlet_Type', data = df, color = 'lightblue'
                  , order = df['Outlet_Type'].value_counts().index)

In [ ]:
null_df = train_df[train_df['Item_Weight'].isnull()]
plot_categorical_column(null_df)

결측치 데이터는 1985년에 설립된 지점들의 데이터들이며, Tier 2 데이터가 없고, Supermarket 1, 2 데이터가 없음

In [ ]:
fig, axes = plt.subplots(3,2, figsize = (18, 25))
fig.suptitle("Bar plot for all categorical variables in the dataset")

sns.countplot(ax = axes[0,0], x = 'Item_Fat_Content', data = train_df, color = 'lightblue'
              , order = train_df['Item_Fat_Content'].value_counts().index)

sns.countplot(ax = axes[0,1], x = 'Item_Type', data = train_df, color = 'lightblue'
              , order = train_df['Item_Type'].value_counts().index)

axes[0,1].tick_params(axis='x', rotation=30, labelsize = 8)

sns.countplot(ax = axes[1,0], x = 'Outlet_Establishment_Year', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Establishment_Year'].value_counts().index)

sns.countplot(ax = axes[1,1], x = 'Outlet_Size', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Size'].value_counts().index)

sns.countplot(ax = axes[2,0], x = 'Outlet_Location_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Location_Type'].value_counts().index)

sns.countplot(ax = axes[2,1], x = 'Outlet_Type', data = train_df, color = 'lightblue'
              , order = train_df['Outlet_Type'].value_counts().index)

특이한 점은 없음

In [ ]:
# fig = plt.figure(figsize = (18,6))

# sns.heatmap(null_df[numerical_column_list].corr(), annot = True)

# plt.xticks(rotation = 45)

In [ ]:
# fig, axes = plt.subplots(1,3, figsize = (20,6))
# fig.suptitle("Bi-variate scatterplot for all numerical variables with the dependent variable")

# for i, col in enumerate(null_df[numerical_column_list].columns):
#     if col == 'Item_Outlet_Sales':
#         continue
#     sns.scatterplot(x = col, y = 'Item_Outlet_Sales', data = null_df, ax = axes[i])

아이템 무게 (수치형 변수) 는 아이템 속성 관련 변수에 따라 달라질 것이다.

속셩 관련 변수는 Item_Fat_Content와 Item_Type이 있다.

In [ ]:
# pd.pivot_table(index = 'Item_Type'
#                , columns = 'Item_Fat_Content'
#                , values = 'Item_Weight'
#                , data = train_df, aggfunc = np.mean)

In [ ]:
train_df[(train_df['Outlet_Establishment_Year'] == 1985)].isnull().sum() / train_df[(train_df['Outlet_Establishment_Year'] == 1985)].shape[0]

1985년 데이터는 아이템 무게가 전부 null이다

In [ ]:
fig = plt.figure(figsize = (18,6))
sns.lineplot(x = 'Outlet_Establishment_Year', y = 'Item_Outlet_Sales', data = train_df, errorbar = ("se", 3), estimator = 'mean')
# estimator : 같은 x축 수준에서 y 변수의 집계 방법, None 이면 모든 관측치를 가지고 옴

위 heatmap을 보면 Item weight는 최소 10에서 14 사이의 평균 값을 가질 것으로 보인다.

아이템 무게가 지점 속성 관련 변수에 따라 달라진다면?

지점 속성 관련 변수는 Outlet Type과 Outlet Location Type 이 있다. (Outlet Size는 Outlet Location Type이랑 관련있을 수 있으므로)

In [ ]:
fig = plt.figure(figsize = (18,6))
sns.lineplot(x = 'Outlet_Establishment_Year', y = 'Item_Outlet_Sales', data = train_df, errorbar = ("se", 3), estimator = 'mean')
# estimator : 같은 x축 수준에서 y 변수의 집계 방법, None 이면 모든 관측치를 가지고 옴

위 heatmap을 보면 Item weight는 13의 평균 값을 가질 것으로 보인다.

In [ ]:
fig = plt.figure(figsize = (18,6))
sns.lineplot(x = 'Outlet_Establishment_Year', y = 'Item_Outlet_Sales', data = train_df, errorbar = ("se", 3), estimator = 'mean')
# estimator : 같은 x축 수준에서 y 변수의 집계 방법, None 이면 모든 관측치를 가지고 옴

위 heatmap을 보면 Item weight는 13의 평균 값을 가질 것으로 보인다.

각 명목형 변수를 쪼개어 보았을 때, 10~14 사이의 값을 가지며 평균 13 정도 값을 가지며,

원 분포 인 uniform distribution을 해치면 안되기 때문에, 10~14 의 난수 값으로 결측치를 대체하자

In [ ]:
item_weight_updated_index = train_df[train_df['Item_Weight'].isnull()].index
train_df.loc[item_weight_updated_index,'Item_Weight'] = np.random.uniform(10, 14, len(item_weight_updated_index))

In [ ]:
item_weight_updated_index = train_df[train_df['Item_Weight'].isnull()].index
train_df.loc[item_weight_updated_index,'Item_Weight'] = np.random.uniform(10, 14, len(item_weight_updated_index))

이제 outlet_size 컬럼의 결측치를 처리할 것인데, 왜 결측치가 생겻는지 부터 데이터를 분리해서 파악해볼 것이다.

In [ ]:
outlet_size_data = train_df[train_df['Outlet_Size'].notnull()]
outlet_size_missing_data = train_df[train_df['Outlet_Size'].isnull()]

In [ ]:
plot_categorical_column(outlet_size_missing_data)

Outlet Size가 결측치인 데이터셋도 Supermarket Type 1 이 가장 많고, Outlet Location Type 은 Tier 2 가 가장 많고 Item Fat Content 는 Low Fat가 가장 많다.

Item_Weight 처럼 특정 년도에 전부 결측치인건 아닌것 같다.

Outlet Size가 결측치 인 데이터들은 지점 속성 데이터에 따라 달라질 것이라면? Outlet Type 에 따라 결측치가 어떻게 다를 것인지 확인해보자

In [ ]:
sns.heatmap(pd.crosstab(index = outlet_size_data['Outlet_Type'], columns = outlet_size_data['Outlet_Size']), annot = True, fmt = 'g')

위 데이터에서 모든 Grocery Store이 Outlet Size가 작은 것을 확인할 수 있고, Supermarket Type 2, 3 은 Outlet Size가 Medium인 것을 확인할 수 있다.

Outlet Location Type에 따라도 Outlet Size가 달라질 수 있을까?

In [ ]:
sns.heatmap(pd.crosstab(index = outlet_size_data['Outlet_Type'], columns = outlet_size_data['Outlet_Size']), annot = True, fmt = 'g')

Outlet Location Type이 Tier 2 인 데이터는 Outlet Size가 모두 Small 이다.

Item 관련 변수와 Outlet Size 변수 간의 데이터를 확인해보면?

In [ ]:
sns.heatmap(pd.crosstab(index = outlet_size_data['Outlet_Type'], columns = outlet_size_data['Outlet_Size']), annot = True, fmt = 'g')

특별한 패턴이 보이지 않는다.

명목형 변수를 대체해야 하는데, High Medium Small중 어떤 값을 고를까?

Outlet size가 null이 아닌 데이터 중 특정 범주에서 Outlet Size값이 하나만 가지는 경우 그 값으로 대체하자

ex

  * Outlet type : Grocery Store -> Outlet Size : Small
  * Outlet type : Supermarket Type2 -> Outlet Size : Medium
  * Outlet type : Supermarket Type3 -> Outlet Size : Medium
  * Outlet Location type : Tier 2 -> Outlet Size : Small

In [ ]:
replace_null_index = outlet_size_missing_data[(outlet_size_missing_data['Outlet_Type'] == 'Grocery Store')].index
train_df.loc[replace_null_index, 'Outlet_Size'] = 'Small'

replace_null_index = outlet_size_missing_data[(outlet_size_missing_data['Outlet_Type'] == 'Supermarket Type2')].index
train_df.loc[replace_null_index, 'Outlet_Size'] = 'Medium'

replace_null_index = outlet_size_missing_data[(outlet_size_missing_data['Outlet_Type'] == 'Supermarket Type3')].index
train_df.loc[replace_null_index, 'Outlet_Size'] = 'Medium'

replace_null_index = outlet_size_missing_data[(outlet_size_missing_data['Outlet_Location_Type'] == 'Tier 2')].index
train_df.loc[replace_null_index, 'Outlet_Size'] = 'Small'

In [ ]:
outlet_size_data = train_df[train_df['Outlet_Size'].notnull()]
outlet_size_missing_data = train_df[train_df['Outlet_Size'].isnull()]

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

# Feature Engineering

데이터 처리 스텝을 완료하였고, 모델링을 하기 전에, 어떤 feature 들이 지금 존재하는 데이터셋의 컬럼들로 만들어질 수 있고, 예측력을 더한는데 필요할지 생각해봐야한다.

지점이 좀 더 오래되었다면, 판매액이 증가했을 것이다 라는 한가지 가설로 시작해 보자. 그럼 오래되었다는 것을 어떻게 정의할 수 있을까? 설립년도라는 컬럼이 2013년도에 수집이 되었다는 것을 알고 있으며, 2013에서 설립년도를 빼면 지점의 설립 경과 년수 (age) 라는 컬럼을 만들 수 있을것 같다.

In [ ]:
train_df["Outlet_Age"] = 2013 - train_df["Outlet_Establishment_Year"]
test_df["Outlet_Age"] = 2013 - train_df["Outlet_Establishment_Year"]

In [ ]:
plt.figure(figsize = (18,6))

sns.boxplot(x = 'Outlet_Age', y = 'Item_Outlet_Sales', data = train_df)

위 시각화를 보면, 설립이 오래되었을 수록 판매액이 증가한다는 사실은 보이지 않는다. 지점의 년도가 다르기 때문에, 판매액은 거의 비슷한 분포를 띄는것 같다. 하지만 이 변수는 저장해 두고, 모델링 빌딩할 때 다시 고려해보고, 변수 중요성을 관찰한 후 나중에 제거하도록 하자.

또다른 가설은 Item_Outlet_Sales / Item_MRP = N_of_Sales_Item 이라는 새로운 변수를 만들고, Item_Type 을 묶어보면 인기있는 상품일 수록 판매액이 높아질 것이고, 년도에 따라 인기있는 상품이 달라지는지 데이터로 확인해 보는것도 의미있을것 같다.

In [ ]:
tmp_df = train_df.copy(deep = True)
tmp_df['N_of_Sales_Item'] = tmp_df['Item_Outlet_Sales'] / tmp_df['Item_MRP']

In [ ]:
sns.histplot(tmp_df['N_of_Sales_Item'], kde = True)

In [ ]:
plt.figure(figsize = (18,6))

sns.boxplot(x = 'Outlet_Age', y = 'Item_Outlet_Sales', data = train_df)

특별히 많이 구매를 한 Item이 있을 줄 알았는데, Item Type에 따라 Sales Item 갯수가 크게 차이나보이진 않는다.

연도별로는 차이가 있을까?

In [ ]:
year_list = sorted(list(tmp_df['Outlet_Establishment_Year'].unique()))

fig, axes = plt.subplots(len(year_list),1, figsize = (18,100))

order_name = sorted(list(tmp_df['Item_Type'].unique()))

for i, year in enumerate(year_list):
    chunk_df = tmp_df[(tmp_df['Outlet_Establishment_Year'] == year)]

    sns.boxplot(x = 'Item_Type', y = 'N_of_Sales_Item', data = chunk_df, ax = axes[i], order = order_name)
    axes[i].set_title(year)
    axes[i].tick_params(labelsize = 10, rotation = 30)

  * 연도별 Item 간에는 판매갯수에 차이는 없어보이나, 년도간에는 Item 갯수에 차이가 있어 보임
  * 1998에는 아이템의 판매갯수가 1~3개 로 다른 년도에 비해 현저히 낮음
  * 1997, 2002, 2004, 2009 년도에는 Seafood가 좀더 많이 팔렸음
  * 해산물 유행 년도 여부 라는 피쳐를 추가하면 판매액이 증가하는 피쳐에 예측력이 높은 변수가 될 수 있을까?
  * 해산물 유행 년도 : 1997, 2002, 2004, 2009년도에 Seafood 아이템 여부

In [ ]:
def create_feature(row):
    if row['Outlet_Establishment_Year'] in (1997, 2002,2004,2009) and row['Item_Type'] == 'Seafood':
        return 'Y'
    else:
        return 'N'

train_df['Seafood_trend_year'] = train_df.apply(lambda x : create_feature(x), axis = 1)
test_df['Seafood_trend_year'] = test_df.apply(lambda x : create_feature(x), axis = 1)

In [ ]:
sns.boxplot(x = 'Seafood_trend_year', y  = 'Item_Outlet_Sales', data = train_df)

Seafood_trend_year 일수록 판매액의 중간값이 높았으나, IQR 범위는 비슷함. -> 모델링에 넣었을 때 유의미한 feature로 나오는지 확인이 필요해 보임

# Modeling

이제, 데이터셋의 변수들에 대하여 분석 해보았으니, 모든 독립변수가 결과변수를 예측하기에 중요한 변수가 아니라는 점을 관찰해보았다. 하지만 처음에는, 모든 변수를 사용해보고 모델 요약을 확인해보면서, 어떤 변수를 모델로 부터 제거할지 결정해볼 것이다. 모델링은 반복적인 업무이다.

In [ ]:
# Outlet_Ages 라는 새로운 변수를 만들어 줬기 때문에 Outlet_Establishment_Year 변수는 drop 할 것이다.
train_features = train_df.drop(['Item_Outlet_Sales', 'Outlet_Establishment_Year'], axis = 1)

train_target = train_df['Item_Outlet_Sales']

우리가 범주형 변수를 독립변수로 사용할 때, 더미 변수라고 알려져있는 원핫 인코딩이 필요합니다.첫번째 카테고리는 drop할 것인데 이것을 reference variable 이라고 알려져 있습니다. 레퍼런스 변수는 선형 회귀식을 해석하는데 도움을 줄것입니다.

In [ ]:
train_features = pd.get_dummies(train_features, drop_first = True)
train_features.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Age,Item_Fat_Content_Regular,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,...,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Seafood_trend_year_Y
0,9.30,0.016047,249.8092,14,False,False,False,False,True,False,...,False,False,True,False,False,False,True,False,False,False
1,5.92,0.019278,48.2692,4,True,False,False,False,False,False,...,True,False,True,False,False,True,False,True,False,False
2,17.50,0.016760,141.6180,14,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
3,19.20,0.000000,182.0950,15,True,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
4,8.93,0.000000,53.8614,26,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False


이제 데이터셋이 같은 범위를 갖기 위해서 수치형 변수들을 정규화해줄 것인데, 우리가 정규화를 하지 않으면, 모델은 높은 범위를 가진 변수에 편향을 가지게 될것이고, 낮은 범위의 변수로 부터는 학습하지 못할 것임. scaling 하기 위한 많은 방법들이 있는데, 여기서는 범주형 변수, 수치형변수 둘다 MinMaxScaler를 사용할 것인데, 범주형 변수에 더미인코딩을 이미 했기때문에 이걸 바꾸진 않을 것임.

In [ ]:
scaler = MinMaxScaler()

train_features_scaled = scaler.fit_transform(train_features)

train_features_scaled = pd.DataFrame(train_features_scaled, index = train_features.index, columns = train_features.columns)

train_features_scaled.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Age,Item_Fat_Content_Regular,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,...,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Seafood_trend_year_Y
0,0.282525,0.048866,0.927507,0.416667,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.081274,0.058705,0.072068,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.770765,0.051037,0.468288,0.416667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.871986,0.000000,0.640093,0.458333,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.260494,0.000000,0.095805,0.916667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
train_features_scaled = sm.add_constant(train_features_scaled) # 판매액은 1원부터 (최소 단위)

In [ ]:
train_features_scaled

,const,Item_Weight,Item_Visibility,Item_MRP,Outlet_Age,Item_Fat_Content_Regular,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Seafood_trend_year_Y
0,1.0,0.282525,0.048866,0.927507,0.416667,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.081274,0.058705,0.072068,0.000000,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.770765,0.051037,0.468288,0.416667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.871986,0.000000,0.640093,0.458333,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.260494,0.000000,0.095805,0.916667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,1.0,0.137541,0.172914,0.777729,0.916667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
8519,1.0,0.227746,0.143069,0.326263,0.291667,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
8520,1.0,0.359929,0.107148,0.228492,0.208333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
8521,1.0,0.158083,0.442219,0.304939,0.000000,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [ ]:
ols_model_0 = sm.OLS(train_target, train_features_scaled)
ols_res_0 = ols_model_0.fit()
print(ols_res_0.summary())

  * R-square가 0.563 정도이다.
  * 모든 변수가 통계적으로 유의미하진 않는다. 모든 독립변수들의 p-value를 확인해보면 통계적으로 유의미한 예측력을 가진 변수들에 대하여 확인해 볼 수 있다.
  * 0.05보다 작은 변수들: 판매가격, Outlet Type : 수퍼마켓 Type 1,2,3, Outlet Size Small, Medium, Outlet Age, Outlet Location Type Tier 3 2 , Item_Type Seafood

In [ ]:
ols_res_0.pvalues.sort_values()

**선형회귀분석 결과해석**

  1. **Adj. R-squared:** 모델 적합도를 설명함
     * Adjust R-sqaured 값은 0~1까지의 값을 가지며, 더 높을 수록 특정 조건이 잘 만족되는 적합이 잘되었음을 나타낸다.
     * 이 케이스 에서는 0.563의 Adjust R-sqaured 값을 가짐.
  2. **coeff** : 한단위의 독립변수가 바뀌었을 때, Y 값의 변화량을 나타낸다. (모든 것은 동일하다고 가정함)
  3. **std err** 계수의 정확도에 대한 수준을 나타냄
     * 낮을 수록 계수가 더욱 정확하다는것을 의미함
  4. **P >|t|** : p-value 값임
     * Pr(>|t|) 는 각 독립 변수의 피쳐들에 대해서 귀무가설과 대립가설이 있음
     * H0 : 독립변수들이 유의미하지 않다.
     * Ha : 독립변수들이 유의미하다.
     * p-value가 0.05보다 작으면, 95% 신뢰수준 하에 통계적으로 유의미하다고 고려됨
  5. **Confidence Interval** 95% 가능도로 계수가 위치할 범위를 나타냄

  * 귀무가설 : Outlet Size와 Item_Outlet_Sales에는 아무런 관계가 없다.



$\mu_{\text {outlet size }=\text { High }}=\mu_{\text {outlet size }=\text { Medium }}=\mu_{\text {outlet size }=\text { Small }}$

where, $\mu$ represents mean sales.

  * 대립 가설 : Outlet Size와 Item_Outlet_Sales에는 무엇인가 관계가 있다.



$\mu_{\text {outlet size = High }} \neq \mu_{\text {outlet size = Medium }} \neq \mu_{\text {outlet size }=\text { small }}$

where, $\mu$ represents mean sales.

위 모델 결과를 보면, p-value가 0.05보다 작기 때문에, 귀무가설을 기각하고 대립가설을 채택할 수 있다.

다름 말로, Outlet Size와 Item_Outlet_Sales간의 관계가 있다는 통계적 증거가 있다고 말할 수 있다.

그래서 p-value가 0.05 이하인 데이터들만 선택할 것이다.

# Feature Selection

## 다중공선성 제거하기

  * 다중공선성은 예측 변수가 상관관계가 있을 때 발생한다. 예측변수는 독립적이어야 되기 때문에 상관관계는 문제가 된다. 만약 독립 변수간의 상관관계가 높다면, 우리가 모델을 적합하고, 결과를 해석할 때 문제를 가져올 수 있다. 선형 모델에서 다중공선성을 가지고 있을 때, 모델이 제안하는 계수는 믿기 어렵다.
  * 다중공선성을 탐색하고 테스트하는 여러 방법들이 있는데, 한가지 방법은 Variation Inflation Factor (분산 팽창 지수)를 이용하는 방법이다.
  * **Variance Inflation factor** (분산 팽창 지수) : 분산 팽창 지수는 얘측기들 사이에 존재하는 공선성 때문에 파라미터가 추정하는 분산의 팽창을 측정한다. 모델의 예측 변수들 사이에 존재하는 상관관계로 인하여 계수인 $\beta_k$ 가 얼마나 팽창되었는지 변동성을 측정하는 지표이다.
  * 일반적인 경험법칙으로 VIF 가 1이면, k번째 예측변수가 다른 예측변수들과 상관관계가 없다고 판단하며, $\beta_k$ 가 팽창하지 않았다고 판단한다. 반면, VIF가 5에 가깝꺼나 5를 초과한다면, 일반적으로 VIF가 10 또는 10을 초과한다면, 높은 다중공선성을 가지고 있다고 본다.

In [ ]:
vif_series = pd.Series(
[variance_inflation_factor(train_features_scaled.values, i ) for i in range(train_features_scaled.shape[1])]
, index = train_features_scaled.columns
, dtype = float )

print("VIF Scores: \n\n{}\n".format(vif_series))

Outlet_Age가 높은 VIF 점수를 가지는 것을 확인해볼 수 있다. 이 변수를 제거하고 모델링을 다시 해보면

In [ ]:
train_features_scaled_new = train_features_scaled.drop("Outlet_Age", axis = 1)

vif_series = pd.Series(
[variance_inflation_factor(train_features_scaled_new.values, i ) for i in range(train_features_scaled_new.shape[1])]
, index = train_features_scaled_new.columns
, dtype = float )

print("VIF Scores: \n\n{}\n".format(vif_series))

In [ ]:
def get_vif_series(dataframe: pd.DataFrame):
    vif_series = pd.Series(
    [variance_inflation_factor(dataframe.values, i ) for i in range(dataframe.shape[1])]
    , index = dataframe.columns
    , dtype = float )

    print("VIF Scores: \n\n{}\n".format(vif_series))

Outlet Age를 제거하고, 회귀 모델을 다시 만들어보면

In [ ]:
ols_model_2 = sm.OLS(train_target, train_features_scaled_new)
ols_res_2 = ols_model_2.fit()
print(ols_res_2.summary())

In [ ]:
ols_res_2.pvalues.sort_values() # 유의미한 변수 11개 -> 7개

더미 변수에 대한 VIF 값은 다른 범주와 상관 관계가 있으므로 일반적으로 높은 VIF를 갖기 때문에 고려하는 것은 좋은 관행이 아니다. p-value를 확인해보면 모두 pvalue가 0.05보다 크기 때문에, Item_Type 컬럼을 drop하고 모델을 다시 building 해볼 것이다.

In [ ]:
drop_columns = [col for col in train_features_scaled_new.columns  if col.startswith('Item_Type')]
drop_columns

In [ ]:
train_features_scaled_new2 = train_features_scaled_new.drop(drop_columns, axis = 1)

get_vif_series(train_features_scaled_new2)

In [ ]:
ols_model_3 = sm.OLS(train_target, train_features_scaled_new2)
ols_res_3 = ols_model_3.fit()
print(ols_res_3.summary())

Item_Weight 가 높은 p-value (0.776) 임을 확인할 수 있음. 그래서 제거 해보고 다시 모델을 빌딩

In [ ]:
drop_columns = [col for col in train_features_scaled_new.columns  if col.startswith('Item_Type')]
drop_columns

In [ ]:
train_features_scaled_new3 = train_features_scaled_new2.drop(drop_columns, axis = 1)

get_vif_series(train_features_scaled_new3)

In [ ]:
ols_model_4 = sm.OLS(train_target, train_features_scaled_new3)
ols_res_4 = ols_model_4.fit()
print(ols_res_4.summary())

Item_Visibility도 높은 pvalue라서 (0.24) 제거

In [ ]:
drop_columns = [col for col in train_features_scaled_new.columns  if col.startswith('Item_Type')]
drop_columns

In [ ]:
train_features_scaled_new4 = train_features_scaled_new3.drop(drop_columns, axis = 1)

get_vif_series(train_features_scaled_new4)

In [ ]:
ols_model_5 = sm.OLS(train_target, train_features_scaled_new4)
ols_res_5 = ols_model_5.fit()
print(ols_res_5.summary())

Outlet Size 관련도 높은 p-value를 가져서 제거

In [ ]:
drop_columns = [col for col in train_features_scaled_new.columns  if col.startswith('Item_Type')]
drop_columns

In [ ]:
train_features_scaled_new5 = train_features_scaled_new4.drop(drop_columns, axis = 1)

get_vif_series(train_features_scaled_new5)

모든 VIF 계수들이 5 이하가 되었다.

In [ ]:
ols_model_6 = sm.OLS(train_target, train_features_scaled_new5)
ols_res_6 = ols_model_6.fit()
print(ols_res_6.summary())

Outlet Location Type 관련 변수도 높은 pvalue 를 가져서 (0.796, 0.687) 제거

In [ ]:
drop_columns = [col for col in train_features_scaled_new.columns  if col.startswith('Item_Type')]
drop_columns

In [ ]:
train_features_scaled_new6 = train_features_scaled_new5.drop(drop_columns, axis = 1)

get_vif_series(train_features_scaled_new6)

In [ ]:
ols_model_7 = sm.OLS(train_target, train_features_scaled_new6)
ols_res_7 = ols_model_7.fit()
print(ols_res_7.summary())

  * VIF score가 5 이하이므로 이 변수들에 대해서는 다중공선성이 없다.
  * 모든 pvalue가 0.05 보다 작으므로, 현재 변수들은 모델에 유의미한 정보를 준다고 볼 수 있다.
  * R-square 값이 그렇게 바뀌지 않았다. 여전히 0.56 정도이다. 이것은 다른 모든 변수들이 모델에 어떤 값들도 주지 않는 다는것을 의미한다.



이제 선형 모델의 가정을 검토해보자.

# 선형 모델의 가정 검토

  1. 잔차 평균은 0 이어야한다.
  2. 오차항이 정규성을 가져야한다.
  3. 변수들이 선형관계여야 한다.
  4. 이분산성이 없어야한다. (등분산성)

## 1\. 잔차 평균이 0 인지

In [ ]:
# Residual
residual = ols_res_7.resid
print(residual.mean())
round(residual.mean(), 7)

0에 근접하므로 가정을 만족한다.

## 2\. 정규성 검증

  * 오차항 (잔차)가 정규분포 여야한다.

  * 만약 오차항이 정규분포를 따르지 않는다면, 신뢰 수준이 너무 넓거나 좁게 나올수 있다. 그래서 신뢰수준이 불안정하면 최소제곱을 최소화 한 것을 기반으로 계수를 추정하기가 어려워진다.

  * 비정규성이 암시하는 것은 무엇일까?

    * 몇몇 비정상적인 데이터포인트가 있으며, 더 낳은 모델을 만들기 위해 면밀히 연구되어야 한다는 것을 의미한다.
  * 정규성을 어떻게 체크할까?

    * 잔차 histogram을 그려봐서 시각적으로 분포를 확인해 볼 수 있다.
    * QQ Plot을 통해서도 확인해 볼 수 있다. 잔차가 정규분포를 따른다면, 일직선 그림을 그릴것이고 그렇지 않으면 일직선이 아닐 것이다.
    * 정규성을 검증하는 방법은 Shapiro-Wilk test가 있다.
  * 잔차가 비정규성이면 어떻게 해야할까?

    * log, 지수, arcsinh 등 데이터에 따라 변형을 해볼 수 있다.

In [ ]:
sns.histplot(tmp_df['N_of_Sales_Item'], kde = True)

잔차가 정규분포인 것을 확인할 수 있으므로 가정을 만족한다.

## 3\. 변수들 간 선형성

예측 변수들간은 종속변수와 선형 관계를 가져야 한다는 것을 의미한다.

이 가정을 테스트 하기 위해서 잔차를 그려보고, 값을 피팅해서 강한 패턴을 형성하지 않는지 확인해봐야한다. 선형성을 가진다면, x 축에 균등하고 임의로 분포되어있을 것이다.

In [ ]:
fitted = ols_res_7.fittedvalues # y_hat

sns.residplot(x = fitted, y = residual, color = 'lightblue')
plt.xlabel("Fitted Values")
plt.ylabel("Residual")
plt.title("Residual PLOT")
plt.show()

핏팅된 값이랑 잔차간에 어떤 관계가 있어보인다. 즉, 잔차가 임의로 분포되어있지 않는 것 같다.

이걸 수정해기 위해, target variable에 log를 취하고 모델을 새롭게 만들어보자

log를 취하는 이유는 ??? -> y값이 클수록 잔차가 커지는 경향이 있어보여서, 값이 커질 수록 줄여주는 스케일링을 하기 위해서

In [ ]:
train_target_log = np.log(train_target)

In [ ]:
ols_model_8 = sm.OLS(train_target_log, train_features_scaled_new6)
ols_res_8 = ols_model_8.fit()
ols_res_8.summary()

OLS Regression Results 
 
 Dep. Variable: Item_Outlet_Sales R-squared: 0.720 
 
 
 Model: OLS Adj. R-squared: 0.720 
 
 
 Method: Least Squares F-statistic: 3653. 
 
 
 Date: Mon, 27 May 2024 Prob (F-statistic): 0.00 
 
 
 Time: 22:31:17 Log-Likelihood: -6810.0 
 
 
 No. Observations: 8523 AIC: 1.363e+04 
 
 
 Df Residuals: 8516 BIC: 1.368e+04 
 
 
 Df Model: 6 
 
 
 Covariance Type: nonrobust 
 
 
 
 
 coef std err t P>|t| [0.025 0.975] 
 
 
 const 4.6360 0.020 234.981 0.000 4.597 4.675 
 
 
 Item_MRP 1.9551 0.022 88.636 0.000 1.912 1.998 
 
 
 Item_Fat_Content_Regular 0.0152 0.012 1.242 0.214 -0.009 0.039 
 
 
 Outlet_Type_Supermarket Type1 1.9535 0.018 109.274 0.000 1.918 1.989 
 
 
 Outlet_Type_Supermarket Type2 1.7710 0.024 73.524 0.000 1.724 1.818 
 
 
 Outlet_Type_Supermarket Type3 2.4837 0.024 103.373 0.000 2.437 2.531 
 
 
 Seafood_trend_year_Y 0.3611 0.099 3.665 0.000 0.168 0.554 
 
 
 
 
 Omnibus: 829.365 Durbin-Watson: 2.008 
 
 
 Prob(Omnibus): 0.000 Jarque-Bera (JB): 1164.634 
 
 
 Skew: -0.775 Prob(JB): 1.27e-253 
 
 
 Kurtosis: 3.936 Cond. No. 23.1 
 
 Notes: [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.

R-square 값이 0.72로 올라갔다.

In [ ]:
# Predicted values
fitted = ols_res_8.fittedvalues

residual1 = ols_res_8.resid

sns.residplot(x = fitted, y = residual1, color = "lightblue")

plt.xlabel("Fitted Values")

plt.ylabel("Residual")

plt.title("Residual PLOT")

plt.show()

White의 이분산검정법

In [ ]:
from statsmodels.stats.diagnostic import het_white

white_test = het_white(ols_res_8.resid, ols_res_8.model.exog)

#define labels to use for output of White's test
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
#print results of White's test
test_result = dict(zip(labels, white_test))
print(test_result)

Test Statistic $X^2$ = 94.36

p-value = 0.000

$H_0$ : 등분산성이 존재한다. (잔차들은 동등하게 흩어져있다)

$H_a$ : 이분산성이 존재한다. (잔차들은 동등하지 않게 흩어져 있다.)

p-value가 0.05 보다 작으므로 귀무가설을 기각하고 이분산성이 존재한다고 봐야함

이번에는 weighted least square regression을 해보자

~~<https://www.geeksforgeeks.org/weighted-least-squares-regression-in-python/>~~

<https://www.statology.org/weighted-least-squares-in-python/>

In [ ]:
weights  = 1/ sm.OLS(ols_res_7.fittedvalues, ols_res_7.resid).fit().fittedvalues ** 2
ols_model_9 = sm.WLS(train_target, train_features_scaled_new6, weights = weights)
ols_res_9 = ols_model_9.fit()
ols_res_9.summary()

WLS Regression Results 
 
 Dep. Variable: Item_Outlet_Sales R-squared: 0.999 
 
 
 Model: WLS Adj. R-squared: 0.999 
 
 
 Method: Least Squares F-statistic: 2.084e+06 
 
 
 Date: Mon, 27 May 2024 Prob (F-statistic): 0.00 
 
 
 Time: 22:31:17 Log-Likelihood: -65233. 
 
 
 No. Observations: 8523 AIC: 1.305e+05 
 
 
 Df Residuals: 8516 BIC: 1.305e+05 
 
 
 Df Model: 6 
 
 
 Covariance Type: nonrobust 
 
 
 
 
 coef std err t P>|t| [0.025 0.975] 
 
 
 const -1375.0523 1.335 -1029.698 0.000 -1377.670 -1372.435 
 
 
 Item_MRP 3667.8773 1.485 2469.565 0.000 3664.966 3670.789 
 
 
 Item_Fat_Content_Regular 49.2726 0.828 59.510 0.000 47.650 50.896 
 
 
 Outlet_Type_Supermarket Type1 1958.0972 0.970 2018.504 0.000 1956.196 1959.999 
 
 
 Outlet_Type_Supermarket Type2 1626.4739 1.069 1522.022 0.000 1624.379 1628.569 
 
 
 Outlet_Type_Supermarket Type3 3364.3614 1.580 2128.798 0.000 3361.263 3367.459 
 
 
 Seafood_trend_year_Y 980.4179 37.016 26.486 0.000 907.858 1052.978 
 
 
 
 
 Omnibus: 29969.958 Durbin-Watson: 2.027 
 
 
 Prob(Omnibus): 0.000 Jarque-Bera (JB): 1417.248 
 
 
 Skew: 0.173 Prob(JB): 1.77e-308 
 
 
 Kurtosis: 1.033 Cond. No. 348. 
 
 Notes: [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.

In [ ]:
# Predicted values
fitted = ols_res_8.fittedvalues

residual1 = ols_res_8.resid

sns.residplot(x = fitted, y = residual1, color = "lightblue")

plt.xlabel("Fitted Values")

plt.ylabel("Residual")

plt.title("Residual PLOT")

plt.show()

잔차 plot이 여전히 관계를 가지는 것으로 보여서 X 설명력은 높아졌지만, 회귀모델을 신뢰할 수 없음

ols_res_8 -> 로 분석을 진행하기로 함

## 이분산성이 아닐 것 (No Heteroscedasticity)

<https://www.statology.org/goldfeld-quandt-test-python/>

등분산성을 테스트하기

  * Homoscedasticity - 잔차의 분산이 회귀선에 따라 동등하게 분포되어있다면, 데이터는 등분산성을 가진다고 말할 수 있다.
  * Heteroscedasticity - 잔차의 분산이 회귀선에따라 동등하지 않다면, 데이터는 이분산성을 가진다고 말할 수 있다. 이런 경우에는, 잔차는 화살표 형상 또는 임의의 다른 비 대칭 형상을 형성할 수 있다.
  * 등분산성 검정을 위해 Goldfeld-Quantdt test를 사용해볼 후 있다. -> 모든 독립변수들에 대하여 일일히 검정
    * 귀무가설 : 잔차가 등분산성이다.
    * 대립가설 : 잔차가 이분산성이다.



<https://m.blog.naver.com/modernyoon/221785691717>

**White test** 정규분포 가정 없이도 수행할 수 있는 일반적인 이분산 검정 방법

**Goldfeld-Quandt test** 오차항의 분산이 독립변수들 중 최소한 한 변수와 강단조 관계에 있다는 가정하에 사용할 수 있는 이분산 검정 방법

In [ ]:
from statsmodels.stats.diagnostic import het_white

white_test = het_white(ols_res_8.resid, ols_res_8.model.exog)

#define labels to use for output of White's test
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
#print results of White's test
test_result = dict(zip(labels, white_test))
print(test_result)

In [ ]:
name = ["F statistic", "p-value"]

for col in train_features_scaled_new6.columns:
    if col == 'const':
        continue

    test = sms.het_goldfeldquandt(train_target_log, train_features_scaled_new6[[col]])
    print(col, lzip(name, test))

~~p-value가 0.05보다 크므로, 귀무가설을 기각할 수 없고, 등분산성을 가진다고 볼 수 잇다.~~

Outlet_Type_Supermarket Type1 변수가 0.05보다 작으므로 귀무가설을 기각하며, 이분산성을 가진다고 볼 수 있다.

In [ ]:
test = sms.het_goldfeldquandt(ols_res_8.resid, ols_res_8.model.exog)
lzip(name, test)

<https://www.statology.org/white-test-in-python/>

In [ ]:
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
white_test = het_white(ols_res_8.resid, ols_res_8.model.exog)
print(dict(zip(labels, white_test)))

het_white test를 해보면, 귀무가설을 기각하여 등분산성을 가진다고 할 수 없고 이분산성을 가진다고 볼 수 있다.

In [ ]:
# weights  = 1/ sm.OLS(ols_res_8.fittedvalues, ols_res_8.resid).fit().fittedvalues ** 2
# ols_model_10 = sm.WLS(train_target_log, train_features_scaled_new6, weights = weights)
# ols_res_10 = ols_model_10.fit()
# ols_res_10.summary()

In [ ]:
# labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
# white_test = het_white(ols_res_10.resid, ols_res_10.model.exog)
# print(dict(zip(labels, white_test)))

In [ ]:
sns.histplot(tmp_df['N_of_Sales_Item'], kde = True)

<https://www.statsmodels.org/stable/glm.html>

일반화 선형 모형

In [ ]:
# import statsmodels.api as sm

# ols_model_11 = sm.GLM(train_target_log, train_features_scaled_new6, family = sm.families.Poisson())
# ols_res_11 = ols_model_11.fit()
# ols_res_11.summary()

In [ ]:
# labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
# white_test = het_white(ols_res_10.resid, ols_res_10.model.exog)
# print(dict(zip(labels, white_test)))

In [ ]:
# from statsmodels.graphics.api import abline_plot

# yhat = ols_res_11.predict()
# y = train_target_log

# fig, ax = plt.subplots()
# ax.scatter(yhat, y, s = 5, marker = 'o', alpha = 0.3)
# line_fit = sm.OLS(y, sm.add_constant(yhat, prepend=True)).fit()
# abline_plot(model_results=line_fit, ax=ax)

# ax.set_title('Model Fit Plot')
# ax.set_ylabel('Observed values')
# ax.set_xlabel('Fitted values');

In [ ]:
# fig, ax = plt.subplots()

# ax.scatter(ols_res_11.predict(), ols_res_11.resid_pearson,  s = 5, marker = 'o', alpha = 0.3)

# # ax.hlines(0, 0, 1)
# # ax.set_xlim(0, 1)
# ax.set_title('Residual Dependence Plot')
# ax.set_ylabel('Pearson Residuals')
# ax.set_xlabel('Fitted values')

In [ ]:
# from scipy import stats

# fig, ax = plt.subplots()

# resid = ols_res_11.resid_deviance.copy()
# resid_std = stats.zscore(resid)
# ax.hist(resid_std, bins=25)
# ax.set_title('Histogram of standardized deviance residuals');

In [ ]:
# from statsmodels import graphics
# graphics.gofplots.qqplot(resid, line='r') # light tailed 분포

변환 (루트/ 세제곱근)

In [ ]:
train_target_sqrt = np.sqrt(train_target)
sns.histplot(train_target_sqrt) # 정규분포와 유사한 분포를 띄는것 같아 이걸로 모델 재 피팅

In [ ]:
ols_model_12 = sm.OLS(train_target_sqrt, train_features_scaled_new6)
ols_res_12 = ols_model_12.fit()
ols_res_12.summary()

OLS Regression Results 
 
 Dep. Variable: Item_Outlet_Sales R-squared: 0.656 
 
 
 Model: OLS Adj. R-squared: 0.656 
 
 
 Method: Least Squares F-statistic: 2706. 
 
 
 Date: Mon, 27 May 2024 Prob (F-statistic): 0.00 
 
 
 Time: 22:31:18 Log-Likelihood: -32349. 
 
 
 No. Observations: 8523 AIC: 6.471e+04 
 
 
 Df Residuals: 8516 BIC: 6.476e+04 
 
 
 Df Model: 6 
 
 
 Covariance Type: nonrobust 
 
 
 
 
 coef std err t P>|t| [0.025 0.975] 
 
 
 const -1.0182 0.395 -2.578 0.010 -1.792 -0.244 
 
 
 Item_MRP 39.0878 0.441 88.535 0.000 38.222 39.953 
 
 
 Item_Fat_Content_Regular 0.4673 0.244 1.913 0.056 -0.011 0.946 
 
 
 Outlet_Type_Supermarket Type1 28.1188 0.358 78.586 0.000 27.417 28.820 
 
 
 Outlet_Type_Supermarket Type2 24.4920 0.482 50.802 0.000 23.547 25.437 
 
 
 Outlet_Type_Supermarket Type3 41.0189 0.481 85.296 0.000 40.076 41.962 
 
 
 Seafood_trend_year_Y 8.9301 1.972 4.529 0.000 5.065 12.795 
 
 
 
 
 Omnibus: 85.884 Durbin-Watson: 2.010 
 
 
 Prob(Omnibus): 0.000 Jarque-Bera (JB): 110.141 
 
 
 Skew: -0.158 Prob(JB): 1.21e-24 
 
 
 Kurtosis: 3.458 Cond. No. 23.1 
 
 Notes: [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.

In [ ]:
# Predicted values
fitted = ols_res_8.fittedvalues

residual1 = ols_res_8.resid

sns.residplot(x = fitted, y = residual1, color = "lightblue")

plt.xlabel("Fitted Values")

plt.ylabel("Residual")

plt.title("Residual PLOT")

plt.show()

In [ ]:
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
white_test = het_white(ols_res_8.resid, ols_res_8.model.exog)
print(dict(zip(labels, white_test)))

<https://www.statology.org/transform-data-in-python/>

In [ ]:
train_target_cbrt = np.cbrt(train_target)
sns.histplot(train_target_cbrt) # 정규분포와 유사한 분포를 띄는것 같아 이걸로 모델 재 피팅

In [ ]:
train_features_scaled_new7 = train_features_scaled_new6.drop(['Item_Fat_Content_Regular'], axis = 1 )

In [ ]:
ols_model_13 = sm.OLS(train_target_cbrt, train_features_scaled_new7)
ols_res_13 = ols_model_13.fit()
ols_res_13.summary()

OLS Regression Results 
 
 Dep. Variable: Item_Outlet_Sales R-squared: 0.683 
 
 
 Model: OLS Adj. R-squared: 0.683 
 
 
 Method: Least Squares F-statistic: 3678. 
 
 
 Date: Mon, 27 May 2024 Prob (F-statistic): 0.00 
 
 
 Time: 22:31:18 Log-Likelihood: -18065. 
 
 
 No. Observations: 8523 AIC: 3.614e+04 
 
 
 Df Residuals: 8517 BIC: 3.618e+04 
 
 
 Df Model: 5 
 
 
 Covariance Type: nonrobust 
 
 
 
 
 coef std err t P>|t| [0.025 0.975] 
 
 
 const 3.1126 0.072 43.110 0.000 2.971 3.254 
 
 
 Item_MRP 7.4626 0.083 90.332 0.000 7.301 7.625 
 
 
 Outlet_Type_Supermarket Type1 5.9793 0.067 89.304 0.000 5.848 6.111 
 
 
 Outlet_Type_Supermarket Type2 5.2834 0.090 58.565 0.000 5.107 5.460 
 
 
 Outlet_Type_Supermarket Type3 8.3029 0.090 92.267 0.000 8.127 8.479 
 
 
 Seafood_trend_year_Y 1.6167 0.369 4.382 0.000 0.894 2.340 
 
 
 
 
 Omnibus: 253.508 Durbin-Watson: 2.011 
 
 
 Prob(Omnibus): 0.000 Jarque-Bera (JB): 290.307 
 
 
 Skew: -0.394 Prob(JB): 9.13e-64 
 
 
 Kurtosis: 3.445 Cond. No. 22.2 
 
 Notes: [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.

In [ ]:
# Predicted values
fitted = ols_res_8.fittedvalues

residual1 = ols_res_8.resid

sns.residplot(x = fitted, y = residual1, color = "lightblue")

plt.xlabel("Fitted Values")

plt.ylabel("Residual")

plt.title("Residual PLOT")

plt.show()

In [ ]:
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
white_test = het_white(ols_res_8.resid, ols_res_8.model.exog)
print(dict(zip(labels, white_test)))

  * 이분산성 자체만으로는 회귀모형에서 bias가 발생하지는 않으나, 예측값의 정확성이 낮아진다고 볼수 있음

  * ols_res_8 이 p-value 가 가장 크기 때문에, ols_res_8 로 분석함

In [ ]:
ols_res_8.params

선형 회귀식은 다음과 같을 것이다.

**log( Item_Outlet_Sales) = 4.64 + 1.96 * Item_MRP$ + 0.02 * Item_Fat_Content_Regular + 1.95 * Outlet_Type_Supermarket Type1 + 1.77 * Outlet_Type_Supermarket Type2 + 2.48 * Outlet_Type_Supermarket Type3 + 0.36 * Seafood_trend_year_Y**

In [ ]:
col_list = list(train_features_scaled.columns)
col_list.remove('const')

In [ ]:
test_features = pd.get_dummies(test_df, drop_first = True)
test_features = test_features[col_list]

test_features_scaled = scaler.transform(test_features)
test_features_scaled = pd.DataFrame(test_features_scaled, columns = col_list)
test_features_scaled = sm.add_constant(test_features_scaled)

test_features_scaled = test_features_scaled[list(train_features_scaled_new6.columns)]
test_features_scaled.head()

,const,Item_MRP,Item_Fat_Content_Regular,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Seafood_trend_year_Y
0,1.0,0.325012,0.0,1.0,0.0,0.0,0.0
1,1.0,0.237819,1.0,1.0,0.0,0.0,0.0
2,1.0,0.893316,0.0,0.0,0.0,0.0,0.0
3,1.0,0.525233,0.0,1.0,0.0,0.0,0.0
4,1.0,0.861381,1.0,0.0,0.0,1.0,0.0


# Evaluation Metrics

## R-Squared

R-squared 지표는 베이스 라인 모델로 부터 우리 모델이 얼마나 좋은지 나타내는 지표입니다.

회귀식이 데이터의 72%를 설명한다는 것을 의미합니다.

In [ ]:
print(ols_res_8.rsquared)

## Mean Squared Error

에러 제곱의 평균을 의미합니다. 즉 추정치와 실제치 사이의 평균 제곱 차이 입니다.

In [ ]:
print(ols_res_8.rsquared)

## Root Mean Squared Error

이 메트릭은 위와 유사하나, 단순히 평균을 취하기 보다 MSE에 루트를 씌워 RMSE 값을 구하며, 타겟 변수와 같은 단위의 메트릭을 구하는데 도움을 줍니다.

In [ ]:
print(np.sqrt(ols_res_8.mse_resid))

모델이 제대로 적합되었는지 corss-validation 점수를 통해 확인해봅니다. **(underfitted, overfitted, right fit)**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lr = LinearRegression()

cv_score1 = cross_val_score(lr, train_features_scaled_new6, train_target_log, cv = 10)

cv_score2 = cross_val_score(lr, train_features_scaled_new6, train_target_log, cv = 10, scoring = 'neg_mean_squared_error')

print("RSquared: %0.3f (+/- %0.3f)" % (cv_score1.mean(), cv_score1.std()*2))

print("Mean Squared Error: %0.3f (+/- %0.3f)" % (-1*cv_score2.mean(), cv_score2.std()*2))

  * cross-validation을 통한 R-squared 가 0.719로 training dataset과 유사함
  * cross-validation을 통한 R-squared 가 0.290로 training dataset과 유사함



모델이 잘 적합했음을 의미함.

# Test dataset Prediction formed

In [ ]:
# These test predictions will be on a log scale
test_predictions = ols_res_8.predict(test_features_scaled)

# We are converting the log scale predictions to its original scale
test_predictions_inverse_transformed = np.exp(test_predictions)

test_predictions_inverse_transformed

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (24, 12))

sns.histplot(test_predictions, ax = ax[0]);

sns.histplot(test_predictions_inverse_transformed, ax = ax[1]);

## 결론

  * 모든 변수들에 대한 단변량 분석, 이변량 분석을 통한 EDA를 수행함
  * 변수들 간의 관계에 기반하여 결측치를 처리함
  * 모든 변수들로 부터 모델링을 수행함
  * 데이터의 다중공선성을 제거하고, 무의미한 변수들을 제거함
  * 선형 회귀의 가정을 확인하고, 가정이 만족되도록 반복적으로 수정
  * 마지막으로 다른 평가지표로 모델을 평가함



제안하는 선형회귀식은 아래와 같음:

**log( Item_Outlet_Sales) = 4.64 + 1.96 * Item_MRP$ + 0.02 * Item_Fat_Content_Regular + 1.95 * Outlet_Type_Supermarket Type1 + 1.77 * Outlet_Type_Supermarket Type2 + 2.48 * Outlet_Type_Supermarket Type3 + 0.36 * Seafood_trend_year_Y**

  * 위 등식으로부터, 판매액을 증가시키기 위해서는 더 높은 MRP item을 더 높은 수퍼마켓 위치에 배치할 필요가 있어 보임 (Type 3 > Type 1 > Type 2) 순
  * 평균적으로 다른 요인들이 모두 같다면, 수퍼마켓 type3 의 판매액의 로그값은 수퍼마켓 type 2 의 판매액의 로그값 의 1.4배 (2.48 / 1.77) 이고, 수퍼마켓 type 1 의 판매액의 로그값의 1.27배 정도 됌



따라서, 위 선형 회귀식을 해석하여, supermarket type 3 가 다른 유형의 지점들 보다 더 많은 판매액에 영향을 준다는 것이 명백하여, 우리는 이들 매장의 매출을 유지 또는 개선하기를 원하고, 나머지 매장의 경우, 예를 들어, 더 나은 고객 서비스를 제공하고, 매장 직원을 위한 더 나은 교육을 제공하고, 높은 MRP 항목에 대한 더 많은 가시성을 제공하는 등 매출을 개선하기 위한 전략을 제시해 볼 수 있음.